In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
#import math 
from pandas.plotting import table
import random
#from itertools import accumulate
import statsmodels.api as sm
# import seaborn as sns
from pandas_datareader.data import DataReader
from regression_output import rd2 as rd
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy import stats
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from tkinter import Label, Button, Entry, OptionMenu, StringVar
import tkinter as tk
from tkinter import ttk
import matplotlib.animation as animation

import matplotlib

from pandas.plotting import register_matplotlib_converters, deregister_matplotlib_converters
register_matplotlib_converters()

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.pyplot import figure as Figure
import matplotlib.pyplot as plt

C:\Users\tytea\OneDrive - Vizio\Documents\Python\marketdata-master\regression_output.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rd2['Price'][rd2['Price'] == inf] = np.nan


              Price  PriceChange  PriceDownFlag
1         62.062500  -148.960807              1
2        114.319626    52.257126              0
3        167.936884    53.617257              0
4        300.000000   132.063116              0
5        287.461538   -12.538462              1
...             ...          ...            ...
129115   728.765957    28.765957              0
129116   848.000000   119.234043              0
129117  1798.000000   950.000000              0
129118  2264.666667   466.666667              0
129119   430.000000 -1834.666667              1

[122456 rows x 3 columns]
AxesSubplot(0.125,0.11;0.775x0.77)


C:\Users\tytea\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [2]:
def regression_formula(X, Y):

    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    Y_pred = linear_regressor.predict(X)  # make predictions
    params = np.append(linear_regressor.intercept_,linear_regressor.coef_)
    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X.reset_index(drop=True)))
    MSE = (sum((Y-Y_pred)**2))/(len(newX)-len(newX.columns))
    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b
    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]
    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)
    df9 = pd.DataFrame()
    df9["Coefficients"],df9["Standard Errors"],df9["t values"],df9["Probabilites"] = [params,sd_b,ts_b,p_values]
    p_values = np.delete(p_values,0,0)
    coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(linear_regressor.coef_))], axis = 1)
    coefficients.columns = ['variable', 'coefficient']                       
    reg = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose([linear_regressor.intercept_]))], axis = 1)
    reg.columns = ['variable', 'intercept']
    pvalues = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(p_values))], axis = 1)
    pvalues.columns = ['variable', 'pvalue']
    m = pd.merge(coefficients, pvalues)
    union = pd.merge(reg, m)
    result = union.fillna(0)
    result['fm'] = "y={}X+{} p={}".format(round(float(result['coefficient']), 3), round(float(result['intercept']), 3), round(float(result['pvalue']),2))
    result = result['fm'].loc[0]
    
    return result

In [3]:
seg_list = rd['UHD Segment'].unique()
size_list = rd['Size (group)'].unique()
brand_list = rd['Brand'].unique()
res_list = rd['Sub Res'].unique()
outlet_list = rd['Outlet'].unique()



In [10]:
matplotlib.use("TkAgg")




#Functions
def close_window (): 
     master.destroy()

def ts1():

    plt.rcParams.update({'figure.figsize':(8,3), 'figure.dpi':100})
    conversion()

    m1 = rd[(rd['Brand'] == conversion.brand) & (rd['Size (group)'] == conversion.size) & (rd['UHD Segment'] == conversion.segment) & (rd['Date'] >= conversion.start_date) & (rd['Date'] <= conversion.end_date) & (rd['Outlet'] == conversion.outlet) & (rd['Sub Res'] == conversion.resolution)]
    m1 = m1.groupby(['Brand', 'Size (group)', 'UHD Segment', 'Date', 'Outlet', 'Sub Res'])['Units'].sum()
    m1 = m1.reset_index(['Date'])
    
    m2 = rd[(rd['Size (group)'] == conversion.size) & (rd['UHD Segment'] == conversion.segment) & (rd['Date'] >= conversion.start_date) & (rd['Date'] <= conversion.end_date) & (rd['Outlet'] == conversion.outlet)]
    m2 = m2.groupby(['Size (group)', 'UHD Segment', 'Date', 'Outlet'])['Units'].sum()
    m2 = m2.reset_index(['Date'])    
    
    m3 = rd[(rd['Size (group)'] == conversion.size) & (rd['UHD Segment'] == conversion.segment) & (rd['Date'] >= conversion.start_date) & (rd['Date'] <= conversion.end_date)]
    m3 = m3.groupby(['Size (group)', 'UHD Segment', 'Date'])['Units'].sum()
    m3 = m3.reset_index(['Date'])  
    
    m4 = rd[(rd['Brand'] == conversion.comp) & (rd['Size (group)'] == conversion.size) & (rd['UHD Segment'] == conversion.segment) & (rd['Date'] >= conversion.start_date) & (rd['Date'] <= conversion.end_date) & (rd['Outlet'] == conversion.outlet) & (rd['Sub Res'] == conversion.resolution)]
    m4 = m4.groupby(['Brand', 'Size (group)', 'UHD Segment', 'Date', 'Outlet', 'Sub Res'])['Units'].sum()
    m4 = m4.reset_index(['Date'])
    
    g1 = plt.plot(m1['Date'], m1["Units"], label=conversion.brand, color='blue')
    g2 = plt.plot(m2['Date'], m2["Units"], label=conversion.outlet, color='green')
    g3 = plt.plot(m3['Date'], m3["Units"], label=conversion.size, color='red')
    g4 = plt.plot(m4['Date'], m4["Units"], label=conversion.comp, color='purple')
    plt.legend(loc='best', numpoints = 1, prop={'size':10})        
        
        
def ts_forecast():

    plt.rcParams.update({'figure.figsize':(8,3), 'figure.dpi':100})
    conversion()
    
    rd1 = rd[(rd['Brand'] == conversion.brand) & (rd['Size (group)'] == conversion.size) & (rd['UHD Segment'] == conversion.segment) & (rd['Date'] >= conversion.start_date) & (rd['Date'] <= conversion.end_date) & (rd['Outlet'] == conversion.outlet) & (rd['Sub Res'] == conversion.resolution)]
    
    md = rd1

    md['Date'] = pd.to_datetime(md['Date'])
    md = md.groupby('Date')['Units', 'Dollars'].sum()
    md = md.reset_index('Date')
    md = md.set_index(['Date'])
    md = md.resample('M').sum()

    md2 = md.reset_index('Date')
    datekeys = md2['Date'].get_values()
    firstdate = datekeys[0]
    lastdate = datekeys[-1]
    md2 = md2[(md2['Date'] != firstdate) & (md2['Date'] != lastdate)]
    md = md2.set_index(['Date'])
    



    model = pm.auto_arima(md.Units, start_p=1, start_q=1,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=3, max_q=3, # maximum p and q
                          m=12,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=True,   # No Seasonality
                          start_P=0, 
                          D=0, 
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)


    # Forecast
    n_periods = 24
    fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
    index_of_fc = np.arange(len(md2.Units), len(md2.Units)+n_periods)

    # make series for plotting purpose
    fc_series = pd.Series(fc, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)

    # Plot
    plt.plot(md2.Units)
    plt.plot(fc_series, color='darkgreen')
    plt.fill_between(lower_series.index, 
                     lower_series, 
                     upper_series, 
                     color='k', alpha=.15)

    plt.title("{} {} ARIMA Forecast".format(conversion.brand, conversion.size))
    plt.show()



def ploting():
    

    
    conversion()

    df2 = rd[(rd['Brand'] == conversion.brand) & (rd['Size (group)'] == conversion.size) & (rd['UHD Segment'] == conversion.segment) & (rd['Date'] >= conversion.start_date) & (rd['Date'] <= conversion.end_date) & (rd['Outlet'] == conversion.outlet) & (rd['Sub Res'] == conversion.resolution)]
    comp = rd[(rd['Brand'] == conversion.comp) & (rd['Size (group)'] == conversion.size) & (rd['UHD Segment'] == conversion.segment) & (rd['Date'] >= conversion.start_date) & (rd['Date'] <= conversion.end_date) & (rd['Outlet'] == conversion.outlet) & (rd['Sub Res'] == conversion.resolution)]
    

    try:
        df3 = df2[np.abs(df2.Productivity-df2.Productivity.mean())<=(3*df2.Productivity.std())] #keep only the ones that are within +3 to -3 standard deviations in the column 'Data'
        X = df3[['Productivity','Price']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
        X = X.loc[:,X.isnull().mean() < .5]
        X = X[~np.isnan(X).any(axis=1)]
        Y = X['Productivity']
        X = X.drop('Productivity', axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        linear_regressor = LinearRegression()  # create object for the class
        linear_regressor.fit(X, Y)  # perform linear regression
        Y_pred = linear_regressor.predict(X)  # make predictions
        y_pred2 = regressor.predict(X_test)
        
        result = regression_formula(X, Y)

        
    except:
        pass
    
    try:
        df7 = comp[np.abs(comp.Productivity-comp.Productivity.mean())<=(3*comp.Productivity.std())] #keep only the ones that are within +3 to -3 standard deviations in the column 'Data'
        Xx = df7[['Productivity','Price']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
        Xx = Xx.loc[:,Xx.isnull().mean() < .5]
        Xx = Xx[~np.isnan(Xx).any(axis=1)]
        Yy = Xx['Productivity']
        Xx = Xx.drop('Productivity', axis=1)
        Xx_train, Xx_test, yy_train, yy_test = train_test_split(Xx, Yy, test_size = 0.3, random_state = 0)
        regressor2 = LinearRegression()
        regressor2.fit(Xx_train, yy_train)
        linear_regressor2 = LinearRegression()  # create object for the class
        linear_regressor2.fit(Xx, Yy)  # perform linear regression
        Yy_pred = linear_regressor2.predict(Xx)  # make predictions
        yy_pred2 = regressor2.predict(Xx_test)
        
        result2 = regression_formula(Xx, Yy)
    except:
        pass

    first = plt.subplot2grid((14,7), (0, 0), rowspan=3, colspan=8)
    first.plot(df3['Date'], df3["Productivity"], label='ST/Loc')
    first.axes.get_xaxis().set_visible(False)
    plt.title('{} {} {} {} ({})  [{} - {}]'.format(e1.get(), e2.get(), e3.get(), e4.get(), e5.get(), e6.get(), e7.get())) 
    plt.legend(loc='upper left', numpoints = 1, prop={'size':7})

    
    top = plt.subplot2grid((14,7), (3, 0), rowspan=2, colspan=8)
    top.plot(df3['Date'], df3["Price"], label='Price', color='blue')
    plt.legend(loc='best', numpoints = 1, prop={'size':7})

    
    bottom = plt.subplot2grid((14,7), (7,0), rowspan=1, colspan=8)
    bottom.bar(df2['Week'], df2['PriceDownFlag'])
    plt.title('PriceDown Flag')
    plt.gcf().set_size_inches(10,10)

    try:
        bottom2 = plt.subplot2grid((14,7), (9,0), rowspan=5, colspan=3)
        bottom2.scatter(X, Y)
        plt.title('{}'.format(e2.get()))
        plt.plot(X, Y_pred, color='blue')
        plt.plot(X_test, y_pred2, color='green', linestyle="dotted")
        plt.ylabel('ST/Loc')
        plt.xlabel('Price')
        table(bottom2, np.round(Y.describe(), 1), loc='best', colWidths=[0.2, 0.2, 0.2])
        plt.text(X.min(),Y.max(),result)
    except:
        pass
    
    try:
        bottom3 = plt.subplot2grid((14,7), (9,4), rowspan=5, colspan=3)
        bottom3.scatter(Xx, Yy)
        plt.title('{}'.format(e8.get()))
        plt.plot(Xx, Yy_pred, color='blue')
        plt.plot(Xx_test, yy_pred2, color='green', linestyle="dotted")
        plt.ylabel('ST/Loc')
        plt.xlabel('Price')
        table(bottom3, np.round(Yy.describe(), 1), loc='best', colWidths=[0.2, 0.2, 0.2])
        plt.text(Xx.min(),Yy.max(),result2)
    except:
        pass


def conversion():

    conversion.outlet = e1.get()
    conversion.brand = e2.get()
    conversion.size = e3.get()
    conversion.segment = e4.get()
    conversion.resolution = e5.get()
    conversion.start_date = datetime.datetime.strptime(e6.get(),'%Y-%m-%d')
    conversion.end_date = datetime.datetime.strptime(e7.get(),'%Y-%m-%d')
    conversion.comp = e8.get()

#-----------------------------------------------------------------------------



master = tk.Tk()
master.title('Market Analysis')
master.geometry('260x260')

Label(master, text="Outlet").grid(row=0)
Label(master, text="Brand").grid(row=1)
Label(master, text="Size").grid(row=2)
Label(master, text="Segment").grid(row=3)
Label(master, text="Resolution").grid(row=4)
Label(master, text="Start Date").grid(row=5)
Label(master, text="End Date").grid(row=6)
Label(master, text="Comp").grid(row=7)

e1 = StringVar(master)
e1.set("WMT001") # default value
e2 = StringVar(master)
e2.set("VIZIO") # default value
e3 = StringVar(master)
e3.set(size_list[4]) # default value
e4 = StringVar(master)
e4.set(seg_list[2]) # default value
e5 = StringVar(master)
e5.set("UHD") # default value
e6 = Entry(master)
e6.insert(10,"2017-01-01")
e7 = Entry(master)
e7.insert(10,"2019-10-29")
e8 = StringVar(master)
e8.set("TCL") # default value


w1 = OptionMenu(master, e1, *outlet_list)
w1.grid(row=0, column=1, sticky='W')
w2 = OptionMenu(master, e2, *brand_list)
w2.grid(row=1, column=1, sticky='W')
w3 = OptionMenu(master, e3, *size_list)
w3.grid(row=2, column=1, sticky='W')
w4 = OptionMenu(master, e4, *seg_list)
w4.grid(row=3, column=1, sticky='W')
w5 = OptionMenu(master, e5, *res_list)
w5.grid(row=4, column=1, sticky='W')
e6.grid(row=5, column=1, sticky='W')
e7.grid(row=6, column=1, sticky='W')
w8 = OptionMenu(master, e8, *brand_list)
w8.grid(row=7, column=1, sticky='W')


Button(master, text='TS Forecast', command=ts_forecast).grid(row=0, column=2, sticky='W', pady=3)
Button(master, text='Regression', command=ploting).grid(row=1, column=2, sticky='W', pady=3)
Button(master, text='TAM', command=ts1).grid(row=2, column=2, sticky='W', pady=3)

master.mainloop()

C:\Users\tytea\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\tytea\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.


Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 12); AIC=626.150, BIC=629.018, Fit time=0.005 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 0, 0, 12); AIC=628.184, BIC=633.920, Fit time=0.042 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 0, 0, 12); AIC=628.126, BIC=632.428, Fit time=0.018 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 0, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 0, 0, 12); AIC=626.230, BIC=630.532, Fit time=0.013 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 12); AIC=625.742, BIC=630.044, Fit time=0.013 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 12); AIC=627.165, BIC=634.335, Fit